In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline
# Below libraries are for feature representation using sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Below libraries are for similarity matrices using sklearn
from sklearn.metrics.pairwise import cosine_similarity
from lightgbm import LGBMClassifier
from sklearn.metrics import pairwise_distances
import copy
from IPython.display import clear_output
#from eli5.lime import TextExplainer
import warnings
#from eli5.lime.samplers import MaskingTextSampler
from re import sub
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
plotly.offline.init_notebook_mode (connected = True)
#import eli5
#from eli5.lime import TextExplainer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from xgboost import XGBClassifier
import random
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
columns = ['NewsID',"Category","SubCategory","Title","Abstract","URL","TitleEntities","AbstractEntities "]
news = pd.read_csv("/content/drive/MyDrive/fairness-mind/MINDsmall_train/news.tsv",sep='\t',header=None, names=columns)

columns =  ["userID", "time", "clickHistory", "ImpLog"]
behavior = pd.read_csv("/content/drive/MyDrive/fairness-mind/MINDsmall_train/behaviors.tsv", sep='\t',header=None, names=columns)

In [ ]:
news.head()

,NewsID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [ ]:
behavior.head()

,userID,time,clickHistory,ImpLog
1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [ ]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51282 entries, 0 to 51281
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NewsID             51282 non-null  object
 1   Category           51282 non-null  object
 2   SubCategory        51282 non-null  object
 3   Title              51282 non-null  object
 4   Abstract           48616 non-null  object
 5   URL                51282 non-null  object
 6   TitleEntities      51279 non-null  object
 7   AbstractEntities   51278 non-null  object
dtypes: object(8)
memory usage: 3.1+ MB


In [ ]:
!pip install plotly

## News Data


Looking at the chart we see: news with titles about 50->100 words long are the most used. This is also a suggestion for writing titles if you want higher interactions

In [ ]:
# Handle empty values
print('Number of posts before filtering :',len(news))
news.drop_duplicates(subset=['Title'],inplace=True)
print('Number of posts after filtering :',len(news))

Number of posts before filtering : 47784
Number of posts after filtering : 47784


In [ ]:
# Handling empty values
news.isnull().sum()

NewsID                  0
Category                0
SubCategory             0
Title                   0
Abstract             2646
URL                     0
TitleEntities           3
AbstractEntities        4
dtype: int64

In [ ]:
# Remove null values ​​in dataframe
news.dropna(inplace=True)
news.head()

,NewsID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [ ]:
news=news.iloc[:,:5]
news.head()

,NewsID,Category,SubCategory,Title,Abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# The function removes stopwords in each specific line and tokenizes them
def rem_stopwords_tokenize(data,name):

    def getting(sen):
        example_sent = sen

        stop_words = set(stopwords.words('english'))

        word_tokens = word_tokenize(example_sent)

        filtered_sentence = [w for w in word_tokens if not w in stop_words]

        filtered_sentence = []

        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        return filtered_sentence
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x

In [ ]:
# lemmatize all words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_all(data,name):
    arr=data[name]
    a=[]
    for i in arr:
        b=[]
        for j in i:
            x=lemmatizer.lemmatize(j,pos='a')
            x=lemmatizer.lemmatize(x)
            b.append(x)
        a.append(b)
    data[name]=a

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Delete Stop words
rem_stopwords_tokenize(news,'Title')

In [ ]:
# Making a copy of data
data=news.copy()

In [ ]:
def convert_to_string(data,name):
    t=data[name].values
    p=[]
    for i in t:
        listToStr = ' '.join(map(str, i))
        p.append(listToStr)
    data[name]=p
convert_to_string(news,'Title')

In [ ]:
print('the number of articles before processing :',len(news))
news=news[news['Title'].apply((lambda x: len(x.split())>=4))]
print('The number of articles after processing :',len(news))

the number of articles before processing : 47784
The number of articles after processing : 47570


In [ ]:
df2 = news.copy()

In [ ]:
headline_vectorizer = TfidfVectorizer()


In [ ]:
def Euclidean_Distance_based_model(row_index, num_similar_items):
    cate=news['Category'][row_index]
    name=news['Title'][row_index]
    cate_data=news[news['Category']==cate]

    row_index2=cate_data[cate_data['Title']==name].index
    headline_features   = headline_vectorizer.fit_transform(cate_data['Title'].values)
    couple_dist = pairwise_distances(headline_features,headline_features[row_index2])
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    df = pd.DataFrame({'headline':df2[df2['Category']==cate]['Title'].values[indices],
                       'Category':cate_data['Category'].values[indices],
                       'Abstract':cate_data['Abstract'].values[indices],
                'Euclidean similarity with the queried article': couple_dist[indices].ravel()})
    print("="*30,"News Article Name","="*30)
    print('News Headline : ',data['Title'][indices[0]])
    print("\n","="*30,"Recommended News : ","="*30)
    return df.iloc[1:,:]
name=input('News Title For Recommendation :')
clear_output()
ind=df2[df2['Title']==name].index[0]
dd=Euclidean_Distance_based_model(ind, 100)
dd.head(10)

============================== News Article Name ==============================
News Headline :  ['50', 'Worst', 'Habits', 'For', 'Belly', 'Fat']

 ============================== Recommended News :  ==============================


,headline,Category,Abstract,Euclidean similarity with the queried article
1,"I Had 1 Cheat Day Once Week For 2 Months , Thi...",health,Some experts and dieters agree that cheat meal...,1.205167
2,'How I Told My Kids About My Breast Cancer ',health,I was scared that they would be scared. I was ...,1.250102
3,Pippa Middleton Takes Baby Son Cranial Osteopa...,health,"Soon after Arthur was born last year, I heard ...",1.265587
4,How Listening My Gut New Symptom Saved My Life,health,A woman with chronic illness describes what ha...,1.276732
5,I Was 24 & Had Just Scored My Dream Job In Fas...,health,One in four adults in the U.S. are living with...,1.278925
6,How Ensure Your Mental Health Remains Priority...,health,"It's commonly referred to as ""the most wonderf...",1.279274
7,"My Doctors Told Me I Had IBS . 4 Years Later ,...",health,"One in three women has this disorder, and it o...",1.283163
8,When I Had Choose Between My Health How I Worship,health,I never thought my chronic illness would force...,1.283797
9,How To Reach Out If Your Friend Is Struggling ...,health,Reaching out to check in on someone who strugg...,1.285456
10,I Have Anxiety Here 's How It Actually Helps Me,health,I was diagnosed with panic disorder 20 years a...,1.286449


In [ ]:
tfidf_headline_vectorizer = TfidfVectorizer(min_df = 0)
def TFIDF_based_model(row_index, num_similar_items):
    cate=news['Category'][row_index]
    name=news['Title'][row_index]
    cate_data=news[news['Category']==cate]

    row_index2=cate_data[cate_data['Title']==name].index
    headline_features   = tfidf_headline_vectorizer.fit_transform(cate_data['Title'].values)
    couple_dist = pairwise_distances(headline_features,headline_features[row_index2])
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    df = pd.DataFrame({'headline':df2[df2['Category']==cate]['Title'].values[indices],
                       'Category':cate_data['Category'].values[indices],
                       'Abstract':cate_data['Abstract'].values[indices],
                'Euclidean Distance Similarity': couple_dist[indices].ravel()})
    print("="*30,"News Article Name","="*30)
    print('News Headline : ',data['Title'][indices[0]])
    print("\n","="*26,"Recommended News Using TFIDf: ","="*30)
    return df.iloc[1:,:]
name=input('News Title For Recommendation :')
clear_output()
ind=df2[df2['Title']==name].index[0]
dd=TFIDF_based_model(ind, 100)
dd.head(10)

============================== News Article Name ==============================
News Headline :  ['50', 'Worst', 'Habits', 'For', 'Belly', 'Fat']

 ========================== Recommended News Using TFIDf:  ==============================


,headline,Category,Abstract,Euclidean Distance Similarity
1,"I Had 1 Cheat Day Once Week For 2 Months , Thi...",health,Some experts and dieters agree that cheat meal...,1.205167
2,'How I Told My Kids About My Breast Cancer ',health,I was scared that they would be scared. I was ...,1.250102
3,Pippa Middleton Takes Baby Son Cranial Osteopa...,health,"Soon after Arthur was born last year, I heard ...",1.265587
4,How Listening My Gut New Symptom Saved My Life,health,A woman with chronic illness describes what ha...,1.276732
5,I Was 24 & Had Just Scored My Dream Job In Fas...,health,One in four adults in the U.S. are living with...,1.278925
6,How Ensure Your Mental Health Remains Priority...,health,"It's commonly referred to as ""the most wonderf...",1.279274
7,"My Doctors Told Me I Had IBS . 4 Years Later ,...",health,"One in three women has this disorder, and it o...",1.283163
8,When I Had Choose Between My Health How I Worship,health,I never thought my chronic illness would force...,1.283797
9,How To Reach Out If Your Friend Is Struggling ...,health,Reaching out to check in on someone who strugg...,1.285456
10,I Have Anxiety Here 's How It Actually Helps Me,health,I was diagnosed with panic disorder 20 years a...,1.286449


## Test some Recommends about the content

---



---



In [ ]:
X=data['Title'].values
y=data['Category'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)

In [ ]:
def print_report(pipe):
    y_pred = pipe.predict(X_test)
    p=np.unique(y_test)
    report = metrics.classification_report(y_test, y_pred,
        target_names=p)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))


### Truncated SVD with Decision Tree Classifier

In [ ]:
vec = TfidfVectorizer(min_df=4)
svd = TruncatedSVD(n_components=400, n_iter=8, random_state=42)
lsa = make_pipeline(vec, svd)
ds=DecisionTreeClassifier()
pipe = make_pipeline(lsa, ds)

In [ ]:
# Assuming X_train and X_test are lists of documents
X_train = [' '.join(document) for document in X_train]
X_test = [' '.join(document) for document in X_test]
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.42836216517857145